In [17]:
import numpy as np
import pandas as pd
import gurobipy as gp
from ast import literal_eval as make_tuple
import time

In [18]:
### Apply for a free Guorbipy Academic Named-User License here: https://www.gurobi.com/features/academic-named-user-license/
### You can find the API parameters of your license here: https://portal.gurobi.com/iam/licenses/list/
PARAMS = {
  "WLSACCESSID": '<WLS ACCESS ID>',
  "WLSSECRET": '<WLS SECRET>',
  "LICENSEID": 0000000,
}

### Choose the relative gap to solve the binary linear program to.
RELATIVE_GAP = 0.01

### Limit the number of degenerate simplex moves, I found setting to 0 helps speed up computation (https://www.gurobi.com/documentation/current/refman/degenmoves.html).
DEGEN_MOVES = 0

### Choose the set of airlines to run the value model for.
### AA - American, AS - Alaskan, B6 - JetBlue, DL - Delta, F9 - Frontier, HA - Hawaiian, NK - Spirit, UA - United, WN - Southwest
AIRLINES = ['AA','AS','B6','DL','F9','HA','NK','UA','WN']

### Choose the set of slot-controlled airports.
### 13930 - Chicago O'Hare, 12892 - Los Angeles, 12478 - NY JFK, 12953 - LaGuardia, 11618 - Newark, 11278 - Washington Reagan, 14771 - San Francisco
### See file airportInfo.csv to add additional airports by their 'AIRPORT_ID'
SLOT_CONTROLLED_AIRPORTS = [13930, 12892, 12478, 12953, 11618, 11278, 14771]

### Create the set of slot allocations. Each should be an integer numpy matrix of size (len(SLOT_CONTROLLED_AIRPORTS),7,48).
### Dim 0 - airports, Dim 1 - days (starting with Monday), Dim 2 - 30 minute time interval over day in airport's local time
SLOT_ALLOCATIONS = [i * np.ones((len(SLOT_CONTROLLED_AIRPORTS),7,48)) for i in range(6)]

In [19]:
def optimal_trimming(slotAllocation, env, numItins, flightsDf, itinDf, itinDict):
  with gp.Model(f'Slot Allocation {airline}', env=env) as m:

    # Initalize timeblocks at 30 minute chunks
    blocks = [100 * j + i for j in range(0, 24) for i in range(29, 60, 30)]

    # Set Gurobipy parameters
    m.setParam(gp.GRB.Param.MIPGap, RELATIVE_GAP)
    m.setParam(gp.GRB.Param.DegenMoves, DEGEN_MOVES)

    # Add decision variables over flights and itinerary choices
    X = m.addVars(flightsDf.shape[0], vtype=gp.GRB.BINARY, name='Flight Decisions')
    Z = m.addVars(numItins, vtype=gp.GRB.BINARY, name='Itinerary Decisions')

    # Constraint 1 - <= 1 itinerary for each passenger
    count = 0
    for key, val in itinDict.items():
      m.addConstr(gp.quicksum(Z[count+i] for i in range(len(val))) <= 1,
                                          name=f'Itinerary Constraint {key}')
      count += len(val)

    # Constraint 2 - itinerary only offered if all flights available
    count = 0
    for key, itins in itinDict.items():
      for i, itin in enumerate(itins):
        for flight in itin:
          m.addConstr(Z[count + i] <= X[flight])
      count += len(itins)

    # Constraint 3 - capacity constraint on flight
    flightsDict = {}
    count = 0
    for key, itins in itinDict.items():
      for i, itin in enumerate(itins):
        for flight in itin:
          if flight not in flightsDict:
            flightsDict[flight] = [(itinDf.iloc[key]['Passengers'],count + i)]
          else:
            flightsDict[flight].append((itinDf.iloc[key]['Passengers'],count + i))
      count += len(itins)

    # Multiply LHS by 10 because we have 10% sampling, multiply RHS by 26 for 26 weeks in season
    for key, itins in flightsDict.items():
        m.addConstr(gp.quicksum(10 * itins[i][0] * Z[itins[i][1]] for i in range(len(itins))) <= 26 * flightsDf.iloc[key]['Seats'],
                                          name=f'Flight Capacity Constraint {key}')


    # Constraint 4 - slot allocation limit
    if slotAllocation is not None:
      slotsDict = {}
      for i, row in flightsDf.iterrows():
        if row.OriginAirportID in SLOT_CONTROLLED_AIRPORTS:
          airport = SLOT_CONTROLLED_AIRPORTS.index(row.OriginAirportID)
          slotInd = blocks.index(row.DepTimeBlk30)
          if (airport, row.DayOfWeek -1, slotInd) in slotsDict:
            slotsDict[(airport, row.DayOfWeek -1, slotInd)].append(i)
          else:
            slotsDict[(airport, row.DayOfWeek -1, slotInd)] = [i]

        if row.DestAirportID in SLOT_CONTROLLED_AIRPORTS:
          airport = SLOT_CONTROLLED_AIRPORTS.index(row.DestAirportID)
          slotInd = blocks.index(row.ArrTimeBlk30)
          if (airport, row.DayOfWeek -1, slotInd) in slotsDict:
            slotsDict[(airport, row.DayOfWeek -1, slotInd)].append(i)
          else:
            slotsDict[(airport, row.DayOfWeek -1, slotInd)] = [i]


      for airport in range(len(SLOT_CONTROLLED_AIRPORTS)):
        for day in range(7):
          for slotInd in range(48):
            if (airport, day, slotInd) in slotsDict:

              m.addConstr(gp.quicksum(X[i] for i in slotsDict[(airport, day, slotInd)]) <= slotAllocation[airport, day, slotInd],
                                              name=f'Slot Constraint {(airport, day, slotInd)}')

    # Constraint 5 - conservation of flow constraint
    aircraftOutDict, aircraftInDict = {}, {}
    for i, row in flightsDf.iterrows():
      if (row.AircraftGroup, row.OriginAirportID) in aircraftOutDict:
        aircraftOutDict[(row.AircraftGroup, row.OriginAirportID)].append(i)
      else:
        aircraftOutDict[(row.AircraftGroup, row.OriginAirportID)] = [i]

      if (row.AircraftGroup, row.DestAirportID) in aircraftInDict:
        aircraftOutDict[(row.AircraftGroup, row.DestAirportID)].append(i)
      else:
        aircraftOutDict[(row.AircraftGroup, row.DestAirportID)] = [i]

    for aircraft, airport in set(aircraftOutDict.keys()).union(set(aircraftInDict.keys())):
      if (aircraft, airport) in aircraftOutDict and (aircraft, airport) in aircraftInDict:
        m.addConstr(gp.quicksum(X[i] for i in aircraftOutDict[(aircraft, airport)]) == gp.quicksum(X[i] for i in aircraftInDict[(aircraft, airport)]), name=f'Flow Constraint {(aircraft, airport)}')
      elif (aircraft, airport) in aircraftOutDict:
        m.addConstr(gp.quicksum(X[i] for i in aircraftOutDict[(aircraft, airport)]) == 0, name=f'Flow Constraint {(aircraft, airport)}')
      else:
        m.addConstr(gp.quicksum(X[i] for i in aircraftInDict[(aircraft, airport)]) == 0, name=f'Flow Constraint {(aircraft, airport)}')

    # Objective Function
    obj = gp.LinExpr()

    count = 0
    for i, row in itinDf.iterrows():
      obj += 10 * row.Passengers * row.Fare * gp.quicksum(Z[count + i] for i in range(len(itinDict[i])))
      count += len(itinDict[i])

    for i, row in flightsDf.iterrows():
      obj -= row.Cost * X[i]

    m.setObjective(obj, gp.GRB.MAXIMIZE)

    m.optimize()


    Xsol = [round(X[i].x) for i in range(flightsDf.shape[0])]
    Zsol = [round(Z[i].x) for i in range(numItins)]



    itinsSat = 0
    count = 0
    for i, row in itinDf.iterrows():
      if sum(Zsol[count:count + len(row)]) > 0:
        itinsSat += 1
      count += len(itinDict[i])


    obj = m.ObjVal
    flightsKept = 100 * (sum(Xsol)/len(Xsol))
    passengersKept = 100 * (itinsSat/numItins)
    solveTime = m.Runtime

    return obj, flightsKept, passengersKept, solveTime

In [ ]:
results = []

for airline in AIRLINES:
  itinDf = pd.read_csv(f'parameters/itineraries/{airline} Itineraries.csv')
  flightsDf = pd.read_csv(f'parameters/weeklyFlights/{airline} Flights.csv')

  itinDict = {}
  for i, row in itinDf.iterrows():
    possItins = []
    possItins.append(make_tuple(row.Itinerary))
    if isinstance(row['Subs Itinerary 1'], str):
      possItins.append(make_tuple(row['Subs Itinerary 1']))
    if isinstance(row['Subs Itinerary 2'], str):
      possItins.append(make_tuple(row['Subs Itinerary 2']))
    if isinstance(row['Subs Itinerary 3'], str):
      possItins.append(make_tuple(row['Subs Itinerary 3']))
    itinDict[i] = possItins

  numItins = sum(len(value) for value in itinDict.values())

  with gp.Env(params=PARAMS,empty=True) as env:
    env.start()
    # None / i=-1 indicates no slot controls, for which results should be measured relative to.
    for i, sa in enumerate(SLOT_ALLOCATIONS + [None]):
      if sa is None:
        i = -1

      print(f'Running slot allocation {i} for airline {airline}')
      profit, flightsKept, passengersKept, solveTime = optimal_trimming(slotAllocation=sa, env=env, numItins=numItins,flightsDf=flightsDf, itinDf=itinDf, itinDict=itinDict)


      results.append([airline, f'Slot Allocation {i}', profit, flightsKept, passengersKept, solveTime])
      print('Results:')
      print('\tProfit: {profit}')
      print(f'% of weekly flights kept: {flightsKept}')
      print(f'% of passengers kept: {passengersKept}')
      print()

    # Solve without slot contraint


pd.DataFrame(results, columns=['Airline', 'Slot Allocation Index', 'Profit', '% of Flights Kept', '% of Passengers Kept', 'Runtime']).to_csv(f'results/{int(time.time())}.csv', index=False)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2403918
Academic license 2403918 - for non-commercial use only - registered to la___@berkeley.edu
Running slot allocation 0 for airline AA
Set parameter MIPGap to value 0.01
Set parameter DegenMoves to value 0
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Academic license 2403918 - for non-commercial use only - registered to la___@berkeley.edu
Optimize a model with 6993806 rows, 3243356 columns and 19975022 nonzeros
Model fingerprint: 0x8a41a04a
Variable types: 0 continuous, 3243356 integer (3243356 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [5e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 4643178 rows and 1982443 columns (pr